In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

test_df = pd.read_csv('test_updated.csv')
train_df = pd.read_csv('train_updated.csv')

target = train_df['RiskFlag']
train_ids = train_df['ProfileID']
test_ids = test_df['ProfileID']

train_features = train_df.drop(columns=['RiskFlag', 'ProfileID'])
test_features = test_df.drop(columns=['ProfileID'])

# Combine for consistent encoding
combined_df = pd.concat([train_features, test_features], axis=0)

# Feature Engineering
# Create ratios that capture risk better than raw numbers
combined_df['Loan_to_Income'] = combined_df['RequestedSum'] / combined_df['AnnualEarnings']
combined_df['EMI_Approx'] = combined_df['RequestedSum'] / combined_df['RepayPeriod']
combined_df['EMI_to_Income'] = combined_df['EMI_Approx'] / (combined_df['AnnualEarnings'] / 12)

# Reduce skewness in financial columns
for col in ['AnnualEarnings', 'RequestedSum']:
    combined_df[col] = np.log1p(combined_df[col])

# Encoding 
education_map = {"High School": 0, "Bachelor's": 1, "Master's": 2, "PhD": 3}
combined_df['QualificationLevel'] = combined_df['QualificationLevel'].map(education_map)

binary_map = {'Yes': 1, 'No': 0}
for col in ['OwnsProperty', 'FamilyObligation', 'JointApplicant']:
    combined_df[col] = combined_df[col].map(binary_map)

combined_df = pd.get_dummies(combined_df, columns=['WorkCategory', 'RelationshipStatus', 'FundUseCase'], drop_first=True)

# Scaling
numeric_cols = combined_df.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
combined_df[numeric_cols] = scaler.fit_transform(combined_df[numeric_cols])

# Polynomial Features 
# Captures non-linear relationships 
# poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
# combined_poly = poly.fit_transform(combined_df)

X = combined_df[:len(train_df)]
X_test_submission = combined_df[len(train_df):]
y = target

In [3]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

svm_model = SVC(kernel='rbf', C=1, gamma='scale', class_weight='balanced', random_state=42)
svm_model.fit(X_train, y_train)

val_preds = svm_model.predict(X_val)

print("Confusion Matrix:\n", confusion_matrix(y_val, val_preds))
print("\nClassification Report:\n", classification_report(y_val, val_preds))

test_preds = svm_model.predict(X_test_submission)

submission = pd.DataFrame({
    'ProfileID': test_ids,
    'RiskFlag': test_preds
})

submission.to_csv('svm_submission.csv', index=False)

Confusion Matrix:
 [[25431 10674]
 [ 1582  3169]]

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.70      0.81     36105
           1       0.23      0.67      0.34      4751

    accuracy                           0.70     40856
   macro avg       0.59      0.69      0.57     40856
weighted avg       0.86      0.70      0.75     40856

